# 敵対的生成ネットワーク(Generative Adversarial Network : GAN)

敵対的生成ネットワーク(GAN)は確率的モデリングの強力なアプローチです(I. Goodfellow et al., 2014; I. Goodfellow, 2016)。
このモデルでは深層生成モデルを家庭し、速く正確な推論が可能になります。

ここではEdwardでの例を示します。原文のWebページ版は　http://edwardlib.org/tutorials/gan　です。

In [1]:
from __future__ import absolute_import
from __future__ import print_function
from __future__ import division

import edward as ed
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
import os
import tensorflow as tf

from edward.models import Uniform
from observations import mnist
from tensorflow.contrib import slim

/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
def plot(samples):
  fig = plt.figure(figsize=(4, 4))
  gs = gridspec.GridSpec(4, 4)
  gs.update(wspace=0.05, hspace=0.05)

  for i, sample in enumerate(samples):
    ax = plt.subplot(gs[i])
    plt.axis('off')
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_aspect('equal')
    plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

  return fig

In [ ]:
ed.set_seed(42)

data_dir = "/tmp/data"
out_dir = "/tmp/out"

M = 128  # バッチサイズ
d = 100  # 隠れ変数の次元

# Data

55,000 枚の$28\times　28$画像で構成されたMNIST(LeCun, Bottou, Bengio, & Haffner, 1998)のデータを使います。
それぞれの画像はflatにされた784次元ベクトルで表現されそれぞれの要素は0から1の間のピクセル値です。

![GAN Fig 0](https://raw.githubusercontent.com/blei-lab/edward/master/docs/images/gan-fig0.png)

このノートでのゴールは高い品質の手書き数字画像を生成するモデルを推定することです。

訓練の間にMNISTの数字のバッチを取り込むことになります。tensorflowのplaceholderを固定したバッチサイズ$M$枚の画像に対してインスタンス化します。

また例となるデータ全体の集合からから次のバッチのデータ点を選び出すヘルパー関数を定義します。
これはある時点で処理中のバッチのインデックス(番号）を保持しておいて次のバッチを関数``next()``を使って返すものです。
推測処理の間`x_train_generator`を使ってバッチを生成することになります。

In [ ]:
def generator(array, batch_size):
  """Generate batch with respect to array's first axis."""
  start = 0  # pointer to where we are in iteration
  while True:
    stop = start + batch_size
    diff = stop - array.shape[0]
    if diff <= 0:
      batch = array[start:stop]
      start += batch_size
    else:
      batch = np.concatenate((array[start:], array[:diff]))
      start = diff
    batch = batch.astype(np.float32) / 255.0  # normalize pixel intensities
    batch = np.random.binomial(1, batch)  # binarize images
    yield batch

In [ ]:
(x_train, _), (x_test, _) = mnist(data_dir)
x_train_generator = generator(x_train, M)
x_ph = tf.placeholder(tf.float32, [M, 784])

# Model

GANは暗黙的な方法で生成モデルを仮定しています。

In [ ]:
def generative_network(eps):
  h1 = slim.fully_connected(eps, 128, activation_fn=tf.nn.relu)
  x = slim.fully_connected(h1, 784, activation_fn=tf.sigmoid)
  return x

with tf.variable_scope("Gen"):
  eps = Uniform(tf.zeros([M, d]) - 1.0, tf.ones([M, d]))
  x = generative_network(eps)

# Inference

In [ ]:
def discriminative_network(x):
  """Outputs probability in logits."""
  h1 = slim.fully_connected(x, 128, activation_fn=tf.nn.relu)
  logit = slim.fully_connected(h1, 1, activation_fn=None)
  return logit

In [ ]:
inference = ed.GANInference(
    data={x: x_ph}, discriminator=discriminative_network)

In [ ]:
optimizer = tf.train.AdamOptimizer()
optimizer_d = tf.train.AdamOptimizer()

inference = ed.GANInference(
    data={x: x_ph}, discriminator=discriminative_network)
inference.initialize(
    optimizer=optimizer, optimizer_d=optimizer_d,
    n_iter=15000, n_print=1000)

In [ ]:
sess = ed.get_session()
tf.global_variables_initializer().run()

idx = np.random.randint(M, size=16)
i = 0
for t in range(inference.n_iter):
  if t % inference.n_print == 0:
    samples = sess.run(x)
    samples = samples[idx, ]

    fig = plot(samples)
    plt.savefig(os.path.join(out_dir, '{}.png').format(
        str(i).zfill(3)), bbox_inches='tight')
    plt.close(fig)
    i += 1

  x_batch = next(x_train_generator)
  info_dict = inference.update(feed_dict={x_ph: x_batch})
  inference.print_progress(info_dict)